In [3]:
# Python script to create train/test/valid
import os
import shutil
import random
original_folder = "/Users/matuskocian/PycharmProjects/vit/data/Training Images/Dry"

# Define paths for the train, test, and validation sets
train_folder = "dataset/train/Dry"
test_folder = "dataset/test/Dry"
valid_folder = "dataset/valid/Dry"

# Define the split ratios
train_ratio = 0.8
test_ratio = 0.2  

# Create directories if they don't exist
for folder in [train_folder, test_folder]:
    os.makedirs(folder, exist_ok=True)

# Get the list of images
images = os.listdir(original_folder)
random.shuffle(images)

# Calculate the number of images for each set
num_images = len(images)
num_train = int(train_ratio * num_images)
num_test = int(test_ratio * num_images)
num_valid = num_images - num_train - num_test

# Copy images to the respective folders
for i, image in enumerate(images):
    src_path = os.path.join(original_folder, image)
    if i < num_train:
        dst_path = os.path.join(train_folder, image)
    else:
        dst_path = os.path.join(test_folder, image)
    shutil.copy(src_path, dst_path)